<a href="https://colab.research.google.com/github/joseortegalabra/Proyect-GANs/blob/master/Crear_videos_pix2pix_nature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#crear frames para videos con red pix2pix
import tensorflow as tf
import os
from matplotlib import pyplot as plt
import numpy as np
import time
from IPython import display
from os import listdir
from os.path import isfile, join
from google.colab import drive
from google.colab import files
from PIL import Image
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#dataset de imagenes de edificios de praga - frontis
class mydata():
  def __init__(self):
    #DATA VIDEO
    !unrar x '/content/drive/My Drive/Proyecto GANs MLBI/dog select.mp4.rar'  
    self.path = '/content/dog select.mp4/'
    path_frame_original = [arch for arch in listdir(self.path) if isfile(join(self.path, arch))] 
    self.path_img = [str(x) + '.jpg' for x in range(len(path_frame_original))]

    #hiperparametros imagenes
    self.img_width = 256   
    self.img_height = 256  

  def load(self, image_file):   
    image = tf.io.read_file(image_file)
    image = tf.image.decode_jpeg(image)    
    w = tf.shape(image)[1]       
    w = w // 2     
    real_image = image[:, :w, :]      
    input_image = image[:, w:, :]      
    input_image = tf.cast(input_image, tf.float32)   
    real_image = tf.cast(real_image, tf.float32)
    return input_image, real_image

  def normalize(self, input_image, real_image):   
    input_image = (input_image / 127.5) - 1
    real_image = (real_image / 127.5) - 1
    return input_image, real_image

  def load_image_test(self, image_file):   
    input_image, real_image = self.load(image_file)  
    input_image, real_image = self.normalize(input_image, real_image)  
    return input_image, real_image
  
  def create_data_video(self, i):
    inp, tar = self.load_image_test(self.path + self.path_img[i])  #image file i
    inp = tf.expand_dims(inp, 0)
    tar = tf.expand_dims(tar, 0)
    return inp, tar

In [ ]:
class pix2pix():
  def __init__(self):
    self.loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)    #crear funcion de perdida binaria para evaluar loss GAN
    
    self.initializer = tf.random_normal_initializer(0., 0.02)     #inicializar los pesos de las capas conv2d, mismo valor que los demas
    
    self.generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)   #definir optimizadores para el generador y el discriminador
    self.discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

    self.epochs = 100    #cantidad de epocas

  def downsample(self, filters, size, apply_batchnorm=True):     
    '''
    crear capa convolusional encoder - reducir dimensionalidad de la imagen.   Crear un modelo secuencial donde solo tiene una capa CONV2D
    Cantidad de filtros y tamaño variable. Stride fijo a 2. Padding = 'same'   -> siempre se reduce a la mitad el tamaño de la imagen (ancho y alto)
    Reducir dimensionalidad usado en generador y discriminador
    (Modelo Secuencial de keras: Sequential groups a linear stack of layers into a tf.keras.Model)
    '''

    result = tf.keras.Sequential()     #Sequential groups a linear stack of layers into a tf.keras.Model. -> Puedo agrupar varias capas de keras y me devuelve las capas unidas y permitiendo unirlas a otras capas de keras
    result.add( tf.keras.layers.Conv2D(filters = filters, kernel_size = size, strides = 2, padding='same', kernel_initializer= self.initializer, use_bias=False))       
    
    if apply_batchnorm:
      result.add(tf.keras.layers.BatchNormalization())   #aplicar batch normalization en el layer de salida

    result.add(tf.keras.layers.LeakyReLU())   #aplicar funcion de activacion

    return result    #retorna el conjunto de capas CONV2D, batch_normalization y activacion unidas y que se pueden unir a otras capas


  def upsample(self, filters, size, apply_dropout=False):
    '''
    crear capa convolusional decoder - aumentar dimensionalidad de la imagen.   Crear un modelo secuencial donde solo tiene una capa CONV2D TRASPUESTA
    Cantidad de filtros y tamaño variable. Stride fijo a 2. Padding = 'same'   -> siempre aumenta al doble el tamaño de la imagen (ancho y alto)
    Aumentar la dimensionalidad usado en generador - Crear imagen ficticia a partir del INPUT y de RUIDO GAUSIANO
    '''  

    result = tf.keras.Sequential()   
    result.add(tf.keras.layers.Conv2DTranspose(filters = filters, kernel_size = size, strides = 2, padding='same', kernel_initializer = self.initializer, use_bias=False))
    result.add(tf.keras.layers.BatchNormalization())   #siempre se aplica batch normalization

    if apply_dropout:
        result.add(tf.keras.layers.Dropout(0.5))   #aplicar dropout con probabilidad 0.5  -> RUIDO GAUSIANO

    result.add(tf.keras.layers.ReLU())

    return result    #return es el modelo creado con la capa convolusion traspuesta con ruido gausiano, batch_normalization y relu

  
  def create_generator(self):  
    inputs = tf.keras.layers.Input(shape=[256,256,3])       #crear layer input de tamaño 256x256x3 obtenido del preprocesamiento

    #definir lista con la capas convolusionales encoder de la U-NET
    down_stack = [
      self.downsample(64, 4, apply_batchnorm=False),         #(None, 128, 128, 64)   cada capa me reduce a la mitad la dimensión
      self.downsample(128, 4),                               #(None, 64, 64, 128)
      self.downsample(256, 4),                               #(None, 32, 32, 256)
      self.downsample(512, 4),                               #(None, 16, 16, 512)
      self.downsample(512, 4),                               #(None, 8, 8, 512)
      self.downsample(512, 4),                               #(None, 4, 4, 512)
      self.downsample(512, 4),                               #(None, 2, 2, 512)
      self.downsample(512, 4),                               #(None, 1, 1, 512)    #reduzo el espacio latente al ancho y alto más pequeño posible  
    ]

    #definir lista con las capas de deconvolusión decoder de la U-NET
    up_stack = [
      self.upsample(512, 4, apply_dropout=True),             # (None, 2, 2, 1024)     #duplico ancho y alto. DUPLICO CANTIDAD DE FILTROS al utilizar los valores del encoder   
      self.upsample(512, 4, apply_dropout=True),             # (None, 4, 4, 1024)      
      self.upsample(512, 4, apply_dropout=True),             # (None, 8, 8, 1024)
      self.upsample(512, 4),                                 # (None, 16, 16, 1024)
      self.upsample(256, 4),                                 # (None, 32, 32, 512)
      self.upsample(128, 4),                                 # (None, 64, 64, 256)
      self.upsample(64, 4),                                  # (None, 128, 128, 128)    #aumento la dimensionalidad hasta el paso anterior a tener la imagen original 256x256
    ]

    #ultima capa decoder -> (None, 256, 256, 3) y ahora con funcion de activacion tangente hiperbolica
    last_decoder = tf.keras.layers.Conv2DTranspose(filters = 3, kernel_size = 4, strides=2, padding='same', kernel_initializer= self.initializer,  activation='tanh') 



    #---->UNIR CAPAS PARA ARMAR GENERADOR U-NET-----<
    x = inputs    #capa de entrada

    #ENCODER
    results_encoder = []    #voy guardando los resultados de las capas de encoder
    for down in down_stack:   #recorro el array con las capas de downsaple
      x = down(x)    #obtengo la salida de pasar por la capa encoder
      results_encoder.append(x)    #guardo los resultados de pasar por cada una de las capas de downsample
    results_encoder = reversed(results_encoder[:-1])    #invierto el orden de los valores y elimino los resultados de la capa (None, 1, 1, 512)    

    #DECODER
    for up, result_encoder in zip(up_stack, results_encoder):    #hago de upsample considerando el valor obtenido al pasar por cada capa encoder y el paso por cada capa decoder
      x = up(x)    #obtengo la salida al pasar por la capa decoder
      x = tf.keras.layers.Concatenate()([x, result_encoder])    #apilar los resultados de downsampling con los de upsampling
    x = last_decoder(x)    #paso por la ultima capa para obtener la imagen deseada que va al discriminador

    self.generator = tf.keras.Model(inputs=inputs, outputs=x)
    


  def create_discriminator(self):
    '''
    Discriminador lee la imagen input y la imagen real y devuelve una matriz 30x30 que me permite clasificar si la imagen es real o falsa
    '''

    input = tf.keras.layers.Input(shape=[256, 256, 3], name='input_image')   #input contorno 
    target = tf.keras.layers.Input(shape=[256, 256, 3], name='target_image')  #imagen real

    #red discriminador
    x = tf.keras.layers.concatenate([input, target])          # (None, 256, 256, channels*2 -> 6)   #unir input y la imagen real
    x = self.downsample(64, 4, apply_batchnorm=False)(x)      # (None, 128, 128, 64)  #crear capa downsample y evaluarla inmediatamente en x input
    x = self.downsample(128, 4)(x)                            # (None, 64, 64, 128)
    x = self.downsample(256, 4)(x)                            # (None, 32, 32, 256) 
    
    #nueva capa zero padding
    x = tf.keras.layers.ZeroPadding2D()(x)                # (None, 34, 34, 256)     agregar vector de ceros a todos los lados
    
    #nueva capa de convlusion downsample con padding valid, batchnormalizacion y activacion leakyrelu      tamaño: [(34 - filtro - 0) / stride]  + 1  
    x = tf.keras.layers.Conv2D(filters = 512, kernel_size = 4, strides= 1, kernel_initializer= self.initializer, use_bias=False)(x)# (None, 31, 31, 512)   
    x = tf.keras.layers.BatchNormalization()(x)   
    x = tf.keras.layers.LeakyReLU()(x)

    #nueva capa zero padding
    x = tf.keras.layers.ZeroPadding2D()(x)                  # (None , 33, 33, 512)

    #nueva capa y ultima convolsion con padding valid, sin batchnormalization y sin activacion ?     tamaño = [(33 - filtro + 0) / stride] + 1
    x = tf.keras.layers.Conv2D(filters = 1, kernel_size = 4, strides=1, kernel_initializer=self.initializer)(x) # (None, 30, 30, 1) 

    #return el model de keras con el input y el output
    self.discriminator = tf.keras.Model(inputs=[input, target], outputs=x)



  def generator_loss(self, discriminator_generated_output, generated_output, real_image):   
    self.LAMBDA = 100    #parametro fijado en el paper - para penalizar norma l1 en la funcion de costos total de la GAN
    '''
    Loss del Generador. 
    gan loss = valor 1 contra el valor obtenida en el discriminador de la imagen obtenida en el generador - que tanto engañe al discriminador con la imagen ficticia
    l1 loss = Imagen real vs imagen obtenida del generador
    Generador busca minimizar su loss, que la imagen ficticia sea clasificada como real.
    '''

    #comparo valor 1 que el discriminador cree que la imagen es real contra el valor obtenido en el discriminador de la imagen fictia
    gan_loss = self.loss_object(tf.ones_like(discriminator_generated_output),   discriminator_generated_output)
    
    #norma 1 entre la imagen real y la imagen obtenida en el generador
    l1_loss = tf.reduce_mean(tf.abs(real_image - generated_output))

    #perdida del generador es la suma de la tipica perdida de la gan más la perdida con la norma l1
    total_gen_loss = gan_loss + (self.LAMBDA * l1_loss)

    return total_gen_loss, gan_loss, l1_loss

  def discriminator_loss(self, discrimator_real_output, discriminator_generated_output):
    '''
    Loss del Discriminador.
    real_loss = valor 1 (es una imagen real) contra valor obtenido al pasar la imagen real por el discriminador
    generated_loss = valor 0  (es una imagen ficticia) contra valor obtenido al pasar la imagen generada por el discriminador.
    Discriminador busca minimizar su loss, que la imagen real sea clasificada como real y que la imagen ficitia sea clasificada como ficticia
    '''
    real_loss = self.loss_object(tf.ones_like(discrimator_real_output), discrimator_real_output)

    generated_loss = self.loss_object(tf.zeros_like(discriminator_generated_output), discriminator_generated_output)

    total_disc_loss = real_loss + generated_loss

    return total_disc_loss


  def generate_images(self, test_input, test_real, save_results = False, index_file = 0): 
    '''
    Funcion creada para mostrar avances de entrenamiento del modelo con una imagen del conjunto de test
    Mostrar input, ground truth, generated
    '''

    #generated_image = self.generator(test_input, training=True)    #especificado por qué le ponen TRUE 
    generated_image = self.generator(test_input, training=False)

    #graficar
    fig = plt.figure(figsize=(15,15))

    #IMAGEN A MOSTRAR DURANTE EL ENTRENAMIENTO ES LA PRIMERA IMAGEN DEL CONJUNTO DE TEST - PERO ESTA VIENE DEL DATASET Y POR LO TANTO REORDENA LA IMAGEN Y SOLO ME ENTREGA UNA IMAGEN CUALQUIERA
    display_list = [test_input[0], test_real[0], generated_image[0]]   #eliminar dimensiones para graficar
    
    title = ['Input Image', 'Ground Truth', 'Generated Image']

    for i in range(3):
      plt.subplot(1, 3, i+1)
      plt.title(title[i])
      # getting the pixel values between [0, 1] to plot it.
      plt.imshow(display_list[i] * 0.5 + 0.5)
      plt.axis('off')
    plt.show()
    
    if(save_results):
      fig.savefig(str(index_file), dpi = 150)

  def generate_images_2(self, test_input, test_real, save_results = False, index_file = 0): 
    generated_image = self.generator(test_input, training=False)

    #graficar
    generated_image_3d = np.squeeze(generated_image)

 
    pil_img = Image.fromarray(((generated_image_3d* 0.5 + 0.5)*255).astype(np.uint8))
    
    #fig = plt.figure(figsize=(15,15))
    #plt.imshow(generated_image_3d * 0.5 + 0.5)
    #plt.axis('off')
    #plt.show()
    
    if(save_results):
      #fig.savefig(str(index_file), dpi = 150, bbox_inches='tight', pad_inches = 0)  
      pil_img.save(str(index_file)+'.png')

  def generate_images_discriminator(self, test_input, test_real, save_results = False, index_file = 0):
    #generated_image = self.generator(test_input, training=True)    #especificado por que le ponen TRUE 
    generated_image = self.generator(test_input, training=False)    #especificado por que le ponen TRUE 
    discriminator_generated_image = self.discriminator([test_input, generated_image], training = False)

    discriminator_ground_truth =  self.discriminator([test_input, test_real], training = False)

    display_list = [discriminator_generated_image, discriminator_ground_truth]
    title = ['Discriminator Generated Image', 'Discriminator Ground Truth']

    fig = plt.figure(figsize=(8,8))
    for i in range(2):
      plt.subplot(2, 1, i+1)
      plt.title(title[i])
      #mostrar resultado discriminador de imagen ficticia y real
      plt.imshow(display_list[i][0,...,-1], vmin=0, vmax=1, cmap='RdBu_r')
      plt.colorbar()
      plt.axis('off')
    #plt.colorbar()
    plt.show()

    if(save_results):
      fig.savefig(str(index_file), dpi = 150)

  def create_checkpoint(self):
    self.checkpoint_dir = './training_checkpoints'
    self.checkpoint_prefix = os.path.join(self.checkpoint_dir, "ckpt")
    self.checkpoint = tf.train.Checkpoint(generator_optimizer=      self.generator_optimizer,
                                          discriminator_optimizer=  self.discriminator_optimizer,
                                          generator    =            self.generator,
                                          discriminator=            self.discriminator)
    
  @tf.function
  def train_step(self, input_image, real_image, num_epoch):
    #entrenar con gradient tape
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      
      #crear imagen ficticia a partir de una imagen de input
      gen_output = self.generator(input_image, training=True)

      #discriminador de la imagen input con la imagen real
      disc_real_output = self.discriminator([input_image, real_image], training=True)
      #discriminador de la imagen input con la imagen generada en el generador
      disc_generated_output = self.discriminator([input_image, gen_output], training=True)

      #calcular la loss del generador 
      gen_total_loss, gen_gan_loss, gen_l1_loss = self.generator_loss(disc_generated_output, gen_output, real_image)
      # calcular la loss de discriminador
      disc_total_loss = self.discriminator_loss(disc_real_output, disc_generated_output)

    
    #---->entrenar la red con los gradientes
    #calcular gradientes del generador
    generator_gradients = gen_tape.gradient(gen_total_loss, self.generator.trainable_variables)
    #calcular gradientes del discriminador
    discriminator_gradients = disc_tape.gradient(disc_total_loss, self.discriminator.trainable_variables)

    #----->aplicar gradiente en el generador
    self.generator_optimizer.apply_gradients(zip(generator_gradients, self.generator.trainable_variables))
    self.discriminator_optimizer.apply_gradients(zip(discriminator_gradients, self.discriminator.trainable_variables))


    #loss en tensor board
    with summary_writer.as_default():
      tf.summary.scalar('gen_total_loss', gen_total_loss, step=num_epoch)
      tf.summary.scalar('gen_gan_loss', gen_gan_loss, step=num_epoch)
      tf.summary.scalar('gen_l1_loss', gen_l1_loss, step=num_epoch)
      tf.summary.scalar('disc_loss', disc_total_loss, step=num_epoch)

  #funcion para entrenar
  def fit(self, train_ds, test_ds):

    for epoch in range(self.epochs):
      start = time.time()
      display.clear_output(wait=True)

      #Ejemplo de una imagen del set de TEST por época
      for example_input, example_target in test_ds.take(1):    #tomo una sola imagen del dataset de test
        self.generate_images(example_input, example_target, save_results = True, index_file = 'Output, epoch: ' +str(epoch))
        self.generate_images_discriminator(example_input, example_target, save_results = True, index_file = 'Output Discriminator, epoch: ' +str(epoch))
      print("Epoch: ", epoch)

      # Entrenar el modelo  - TRAIN
      for n, (input_image, target) in train_ds.enumerate():
        print('.', end='')    #barra progreso
        if (n+1) % 100 == 0:
          print()
        self.train_step(input_image, target, epoch)  #entrenar la red con BATCH TAMAÑO 1
      print()

      # guardar checkpoint del modelo cada 20 epocas
      if (epoch + 1) % 20 == 0:
        self.checkpoint.save(file_prefix = self.checkpoint_prefix)

      print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1, time.time()-start))
    self.checkpoint.save(file_prefix = self.checkpoint_prefix)

model_gan = pix2pix()     #crear objeto modelo
model_gan.create_generator()    #crer modelo generador
model_gan.create_discriminator()   #crear modelo discriminador

### Load CK



In [ ]:
#restoring the latest checkpoint in checkpoint_dir
path_ck = '/content/drive/My Drive/CK pix2pix animals (30%)/'   #directorio del ck a cargar
model_gan.create_checkpoint()
model_gan.checkpoint.restore(tf.train.latest_checkpoint(path_ck))

### DESCARGAR FRAMES DEL VIDEO

In [ ]:
data_video = mydata()


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/drive/My Drive/Proyecto GANs MLBI/dog select.mp4.rar

Creating    dog select.mp4                                            OK
Extracting  dog select.mp4/0.jpg                                           0%  OK 
Extracting  dog select.mp4/1.jpg                                           1%  OK 
Extracting  dog select.mp4/10.jpg                                          1%  OK 
Extracting  dog select.mp4/100.jpg                                         2%  OK 
Extracting  dog select.mp4/101.jpg                                         2%  OK 
Extracting  dog select.mp4/102.jpg                                         3%  OK 
Extracting  dog select.mp4/103.jpg                                         3%  OK 
Extracting  dog select.mp4/104.jpg                                         4%  OK 
Extracting  dog select.mp4/105.jpg         

In [ ]:
from zipfile import ZipFile
zipObj = ZipFile('video_1.zip', 'w')
os.mkdir('video_1/')

In [ ]:
for i in range(len(data_video.path_img)):
#for i in range(3):
  input, target = data_video.create_data_video(i)
  model_gan.generate_images_2(input, target, save_results= True,    index_file = 'video_1/' + str(i) )
  zipObj.write('video_1/' + str(i) + '.png')
  #files.download('video_1/' + str(i) + '.png')   #guardar imagen por imagen
zipObj.close()
files.download('video_1.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>